# PayPal Requirements

##### Describes requirments from PayPal story that can be found here: [PayPal Story](https://docs.google.com/document/d/1oVM5ewbC3UVdr21dlR3kjD5pRChdSm7y/edit)

---

## Connect to the cluster

Run this command in separate terminal to forward port from Katana controller node

```gcloud compute ssh --zone us-east1-b --project CLUSTER_NAME katana-controller -- -NL 8080:127.0.0.1:8080 -vvv ```

Connect local Docker container to Katana Controller

In [ ]:
import os
from timeit import default_timer as timer

import pandas as pd
from katana import remote
from katana.remote import analytics, import_data

# os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

NUM_PARTITIONS = 20

print("--")


## Initial Load - Social Graph (100mm nodes, 19bn edges) (demo'd this on Jun 16 to PayPal)

Specify input and output locations on cloud storage

In [ ]:
graph = remote.Client().create_graph(num_partitions=NUM_PARTITIONS)

print("--")


In [ ]:

start = timer()

print("Importing graph from CSV files...")

import_data.csv(
    graph,
       #
    input_node_path  = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/node_list_blacklist0.01.txt",
    input_edge_path  = "gs://katana-demo-datasets/csv-datasets/social/mapping_files/edge_list_full.txt",
    input_dir        = "gs://katana-demo-datasets/csv-datasets/social/csv/100_Million",
       #
    data_delimiter   = "|",
    schema_delimiter = "|",
    ids_are_integers = True,
)
end = timer()
print(f"  import: {end - start:.1f} seconds")

start = timer()
assert graph.num_nodes() == 100_000_000
print(f"  num nodes: {graph.num_nodes()}")
end = timer()
print(f"  count nodes: {end - start:.1f} seconds")

start = timer()
assert graph.num_edges() == 19_438_215_362
print(f"  num nodes: {graph.num_edges()}")
end = timer()
print(f"  count edges: {end - start:.1f} seconds")

#  Farrell,
#
#     .  Import takes 41 mins
#     .  Count nodes takes  81 seconds
#     .  Count edges takes 157 seconds


In [ ]:
start = timer()

query1 = """

   match (acct1)-[:FriendConB]->(acct2)-[:FriendConB]->(acct3)
   where (acct3.userIsBlacklisted="Y" or acct3.userIsSuspended="Y")
   and not (acct1)-->(acct3)
   return acct1.id, count(acct3) ;

"""

query1_result = graph.query_paginated(query1)
end = timer()
print(query1_result.head())
print(f"  query1: {end - start:.1f} seconds")

#  Farrell,
#
#     .  After many minutes (20-40 ?), got this error,
#           OperationError: EY8Rb8NZ68Z2yJhi1sFXb5opkZSmrHY7GooZ4MwmGGHc-2FtAJzHrFXgmJ2Rro 
#
#        Run Id,   2817714824
#        node_count : 20
#        machine_type : "n2-highmem-32"


In [ ]:

#  Full error,
#
#  ---------------------------------------------------------------------------
#  OperationError                            Traceback (most recent call last)
#  Input In [4], in <cell line: 12>()
#        1 start = timer()
#        3 query1 = """
#        4 
#        5    match (acct1)-[:FriendConB]->(acct2)-[:FriendConB]->(acct3)
#     (...)
#        9 
#       10 """
#  ---> 12 query1_result = graph.query_paginated(query1)
#       13 end = timer()
#       14 print(query1_result.head())
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:249, in AsyncToSync.<locals>.do_wrap.<locals>.wrapper(self, *args, **kwargs)
#      246 @wraps(underlying_func)
#      247 def wrapper(self, *args, **kwargs):
#      248     return registry.async_to_sync(
#  --> 249         underlying_func(
#      250             get_self_func(self),
#      251             *(registry.sync_to_async(a) for a in args),
#      252             **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
#      253         )
#      254     )
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:176, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
#      166     registry = AsyncToSyncClassRegistry.get()
#      167     return registry.async_to_sync(
#      168         wait_for(
#      169             async_func(
#     (...)
#      174         )
#      175     )
#  --> 176 return wait_for(async_func(*args, **kwargs), timeout=timeout)
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
#      145 try:
#      146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
#  --> 147     return future.result()
#      148 except KeyboardInterrupt:
#      149     inner_future.cancel()
#  
#  File /opt/conda/lib/python3.8/concurrent/futures/_base.py:444, in Future.result(self, timeout)
#      442     raise CancelledError()
#      443 elif self._state == FINISHED:
#  --> 444     return self.__get_result()
#      445 else:
#      446     raise TimeoutError()
#  
#  File /opt/conda/lib/python3.8/concurrent/futures/_base.py:389, in Future.__get_result(self)
#      387 if self._exception:
#      388     try:
#  --> 389         raise self._exception
#      390     finally:
#      391         # Break a reference cycle with the exception in self._exception
#      392         self = None
#  
#  File /opt/conda/lib/python3.8/asyncio/tasks.py:455, in wait_for(fut, timeout, loop)
#      450     warnings.warn("The loop argument is deprecated since Python 3.8, "
#      451                   "and scheduled for removal in Python 3.10.",
#      452                   DeprecationWarning, stacklevel=2)
#      454 if timeout is None:
#  --> 455     return await fut
#      457 if timeout <= 0:
#      458     fut = ensure_future(fut, loop=loop)
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:334, in Graph.query_paginated(self, query, memory_usage_factor, **parameters)
#      331 if memory_usage_factor:
#      332     parameters["__katana_internal_match_batch_limit_scale_factor"] = memory_usage_factor
#  --> 334 res = await self._run_query(query, parameters=parameters, parquet=True)
#      335 result = PaginatedResultSet(self, res["result_files"], res["operation_id"])
#      336 await self._client._refresh_graph_version(self)
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:276, in Graph._run_query(self, query, parameters, parquet)
#      272     data["cypher"]["output_type"] = "Parquet"
#      274 data = self._add_operation_metadata(data)
#  --> 276 res = await self._client._run_on_graph_and_wait(self, data)
#      278 return res
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:310, in Database._run_on_graph_and_wait(self, graph, data)
#      308 async def _run_on_graph_and_wait(self, graph: Graph, data) -> dict:
#      309     self._check_graph_from_myself(graph)
#  --> 310     return await self._wait_op(await self._run_on_graph(graph, data))
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:252, in Database._wait_op(self, op)
#      250 async with _progress_bar() as pbar:
#      251     async with wait_fn(operation_id) as stream:
#  --> 252         async for update in stream:
#      253             status = update["status"]
#      255             for entry in op.get("progress", []):
#  
#  File /opt/conda/lib/python3.8/site-packages/asyncstdlib/builtins.py:445, in map(function, *iterable)
#      443 async for args in args_iter:
#      444     result = function(*args)
#  --> 445     yield await result
#  
#  File /opt/conda/lib/python3.8/site-packages/asyncstdlib/_core.py:134, in force_async.<locals>.async_wrapped(*args, **kwargs)
#      133 async def async_wrapped(*args: Any, **kwargs: Any) -> T:
#  --> 134     return call(*args, **kwargs)
#  
#  File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:142, in _OperationClient._event_stream.<locals>.parse_stream(line)
#  #      139 status = data["status"]
#      141 if status == "Failed":
#  --> 142     raise errors.OperationError(operation_id + " " + data["status_message"])
#      143 if status == "Canceled":
#      144     raise errors.CanceledError()
#  
#  OperationError: EY8Rb8NZ68Z2yJhi1sFXb5opkZSmrHY7GooZ4MwmGGHc-2FtAJzHrFXgmJ2Rro 
        

Report requirements: 
- Cluster size and SKU used for data ingestion
- % of completion
- Time used for ingestion
- Warnings & errors happened so far
- Statistics of # of vertices and edges successfully ingested & not ingested due to errors, etc.



Output: 
- Report already shows skew (complete - available now)
- Cluster size and SKU used for data ingestion (complete - available now)
- Time used for ingestion (complete - available now)
- % of completion
- Warnings & errors happened so far (as the load is occurring..) 
- Statistics of # of vertices and edges successfully ingested & not ingested due to errors, etc. (complete - available now) 


## Incremental Data Loading - Social Graph 

Requirements: (data will be staged in gs) 
- 80K of vertices to be inserted
- 7M of edges to be inserted


In [ ]:
# incremental node+edge inserts, these should also work separately by leading the node or edge mapping path blank

next_node_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/node_insert_100M_10M.txt"
)
next_edge_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/edge_insert_100M_7M.txt"
)

start = timer()
# re-uses the graph from initial load
import_data.csv(
    graph,
    operation=import_data.Operation.Insert,
    input_node_path=next_node_ingest_batch,
    input_edge_path=next_edge_ingest_batch,
    input_dir="gs://katana-demo-datasets/csv-datasets/social/csv/",
    data_delimiter="|",
    schema_delimiter="|",
    ids_are_integers=True,
)

end = timer()
print(f"  Insert Op: {end - start:.1f} seconds")

- 100K of vertices to be updated
- 1M of edges to be updated


In [ ]:
# incremental node + edge updates (XXX these paths are guesses)

next_node_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/node_update_100M_10M.txt"
)
next_edge_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/edge_update_100M_10M.txt"
)

start = timer()
# re-uses the graph from initial load
import_data.csv(
    graph,
    operation=import_data.Operation.Update,
    input_node_path=next_node_ingest_batch,
    input_edge_path=next_edge_ingest_batch,
    input_dir="gs://katana-demo-datasets/csv-datasets/social/csv/",
    data_delimiter="|",
    schema_delimiter="|",
    ids_are_integers=True,
)

end = timer()
print(f"  Update Op: {end - start:.1f} seconds")

- 100K of vertices to be deleted
- 1M of edges to be deleted


In [ ]:
# incremental node + edge deletes (XXX these paths are guesses)

next_node_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/node_delete_100M_10M.txt"
)
next_edge_ingest_batch = (
    "gs://katana-demo-datasets/csv-datasets/social/mapping_files/incremental/edge_delete_100M_10M.txt"
)

start = timer()
# re-uses the graph from initial load
import_data.csv(
    graph,
    operation=import_data.Operation.Delete,
    input_node_path=next_node_ingest_batch,
    input_edge_path=next_edge_ingest_batch,
    input_dir="gs://katana-demo-datasets/csv-datasets/social/csv/",
    data_delimiter="|",
    schema_delimiter="|",
    ids_are_integers=True,
)

end = timer()
print(f"  Delete Op: {end - start:.1f} seconds")

Output: 
- Report already shows skew (complete - available now)
- Cluster size and SKU used for data ingestion (complete - available now)
- Time used for ingestion (complete - available now)
- % of completion
- Warnings & errors happened so far (as the load is occurring..) 
- Statistics of # of vertices and edges successfully ingested & not ingested due to errors, etc. (complete - available now) 


## f) Query execution and output

#### Run 2hop aggregations only on edges of type FriendConB: for each account, find all his neighbors within 2 hops exactly (exclude accounts that are 1 hop from the account) and do the following aggregations on all these accounts: 
sum(user_is_blacklisted=’Y’ or suspended=’Y’)/(# of accounts in 2 hops)
sum(phone_is_verified=’Y’) /(# of accounts in 2 hops)
avg(cnt_decl)

##### FriendB Connection Only
- 2 hop query by account
- Global triangles for totals
- Users suspended and blacklisted
- Users not verified
- Average count of users declined

In [ ]:
## Corresponds https://github.com/KatanaGraph/katana-tools/blob/main/bench/queries/paypal/11.q used by QA
start = timer()

query2 = """
MATCH (acct1)-[:FriendConB]->(acct2)-[:FriendConB]->(acct3) 
WHERE NOT (acct1)-[:FriendConB]->(acct3) 
WITH acct1, avg(acct3.cntDecl) as avgCntDecl, count(acct3) as totalAccounts 
MATCH (acct1)-[:FriendConB]->(acct2)-[:FriendConB]->(acct3Bad) 
WHERE (acct3Bad.userIsBlacklisted="Y" OR acct3Bad.userIsSuspended="Y") 
AND NOT (acct1)-[:FriendConB]->(acct3Bad) 
WITH acct1, avgCntDecl, totalAccounts, count(acct3Bad) as suspAccounts 
MATCH (acct1)-[:FriendConB]->(acct2)-[:FriendConB]->(acct3Bad) 
WHERE acct3Bad.phoneIsVerified="N" 
AND NOT (acct1)-[:FriendConB]->(acct3Bad) 
WITH acct1, avgCntDecl, totalAccounts, suspAccounts, count(acct3Bad) as unverifiedAccounts 
RETURN acct1.id, avgCntDecl, toFloat(suspAccounts) / toFloat(totalAccounts) as percentSusp, toFloat(unverifiedAccounts) / toFloat(totalAccounts) as percentUnverified;
"""

query2_result = graph.query_paginated(query2)
end = timer()
print(query2_result.head())
print(f"  query2: {end - start:.1f} seconds")

Output: 
- Report already shows skew (complete - available now)
- Cluster size and SKU used for data ingestion (complete - available now)
- Time used for ingestion (complete - available now)
- % of completion
- Warnings & errors happened so far (as the load is occurring..) 
- Statistics of # of vertices and edges successfully ingested & not ingested due to errors, etc. (complete - available now) 


#### Run 2hop aggregations on the full graph: for each account, find all his neighbors within 2 hops exactly (exclude accounts that are 1 hop from the account) and do the following aggregations on all these accounts: 
sum(user_is_blacklisted=’Y’ or suspended=’Y’)/(# of accounts in 2 hops)
sum(phone_is_verified=’Y’) /(# of accounts in 2 hops)
avg(cnt_decl)

##### Full Social Graph
- 2 hop query by account
- Global triangles for totals
- Users suspended and blacklisted
- Users not verified
- Average count of users declined

In [ ]:
## Corresponds https://github.com/KatanaGraph/katana-tools/blob/main/bench/queries/paypal/12.q used by QA
start = timer()

query3 = """
MATCH (acct1)-->(acct2)-->(acct3) 
WHERE NOT (acct1)-->(acct3) 
WITH acct1, avg(acct3.cntDecl) as avgCntDecl, count(acct3) as totalAccounts 
MATCH (acct1)-->(acct2)-->(acct3Bad) 
WHERE (acct3Bad.userIsBlacklisted="Y" OR acct3Bad.userIsSuspended="Y") 
AND NOT (acct1)-->(acct3Bad) 
WITH acct1, avgCntDecl, totalAccounts, count(acct3Bad) as suspAccounts 
MATCH (acct1)-->(acct2)-->(acct3Bad) 
WHERE acct3Bad.phoneIsVerified="N" 
AND NOT (acct1)-->(acct3Bad) 
WITH acct1, avgCntDecl, totalAccounts, suspAccounts, count(acct3Bad) as unverifiedAccounts 
RETURN acct1.id, avgCntDecl, toFloat(suspAccounts) / toFloat(totalAccounts) as percentSusp, toFloat(unverifiedAccounts) / toFloat(totalAccounts) as percentUnverified;
"""

query3_result = graph.query_paginated(query3)
end = timer()
print(query3_result.head())
print(f"  query3: {end - start:.1f} seconds")

##### Full Social Graph
- 2 hop query by account
- Global triangles for totals
- Users suspended and blacklisted
- Users not verified
- Average count of users declined

## Algorithms

#### PageRank (Social Graph)

In [ ]:
analytics.pagerank(graph, result_property_name="page_rank")
result = graph.query(
    """
    MATCH (n)
    RETURN n, n.page_rank
    ORDER BY n.page_rank DESC
    LIMIT 30
    """,
    contextualize=True,
)
pd.DataFrame(result[0:10])

#### Distributed Louvain (Social Graph)

In [ ]:
prop_name = "cluster_id"
louvain_graph = graph.project(node_types=["Account"], edge_types=["AssetConB"])
analytics.louvain_clustering(louvain_graph, result_property_name=prop_name, is_symmetric=True)
result = graph.query(
    f"""
    MATCH (n)
    WHERE exists(n.{prop_name})
    RETURN n, n.{prop_name}
    LIMIT 30
    """,
    contextualize=True,
)
pd.DataFrame(result[0:10])

#### Scale to 20B nodes and 40B edges and execute queries and analytics 